In [11]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [12]:
class MyDataset(Dataset):
    # 构造函数
    def __init__(self, data_tensor, target_tensor):
        self.data_tensor = data_tensor
        self.target_tensor = target_tensor
    # 返回数据集大小
    def __len__(self):
        return self.data_tensor.size(0)
    # 返回索引的数据与标签
    def __getitem__(self, index):
        return self.data_tensor[index], self.target_tensor[index]

In [13]:
from scipy.io import loadmat
file6 = "D:\\320 Deep-learning\data\\1118_S6_time1000.mat"
data6 = loadmat(file6)
file12 = "D:\\320 Deep-learning\data\\1111_S12_time1000.mat"
data12 = loadmat(file12)
file24 = "D:\\320 Deep-learning\data\\1111_S24_time1000.mat"
data24 = loadmat(file24)
""" experiment
X_train = data['Input']
X_train = torch.from_numpy(X_train).float()
y_train = data['Stability']
y_train = y_train.astype(np.float32) / 255
y_train = torch.tensor(y_train)
len_train = len(y_train)
X_test = data['PreInput']
X_test = torch.from_numpy(X_test).float()
y_test = data['PreSta']
y_test = y_test.astype(np.float32) / 255
y_test = torch.tensor(y_test)
len_test = len(y_test) """

#simulation
X6 = data6['Input_Abundance']
X6 = torch.from_numpy(X6).float()
y6 = data6['Stability']
y6 = torch.from_numpy(y6).float()

X12 = data12['Input_RelativeAbundance']
X12 = torch.from_numpy(X12).float()
y12 = data12['Stability']
y12 = torch.from_numpy(y12).float()

X24 = data24['Input_Presence']
X24 = torch.from_numpy(X24).float()
y24= data24['Stability']
y24 = torch.from_numpy(y24).float()

t = 500
X_train = X24[0:t,:]
y_train = y24[0:t]
X_test = X24[t:1000,:]
y_test = y24[t:1000]

len_train = len(y_train)
len_test = len(y_test)

# total number of 0 or 1 in training data (prepare for weight)
Num_0 = (y_train==0).sum()
Num_1 = (y_train==1).sum()
Weights = torch.Tensor([Num_0.item()/Num_0.item(),Num_0.item()/Num_1.item()])


# 将数据封装成Dataset
train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

In [14]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 10

# prepare data loaders
train_dataloader = DataLoader(dataset = train_dataset, # 传入的数据集, 必须参数
                               batch_size = batch_size,       # 输出的batch大小
                               shuffle = True,       # 数据是否打乱
                               num_workers = num_workers)      # 进程数, 0表示只有主进程
test_dataloader = DataLoader(dataset = test_dataset, # 传入的数据集, 必须参数
                               batch_size = batch_size,       # 输出的batch大小
                               shuffle = True,       # 数据是否打乱
                               num_workers = num_workers)      # 进程数, 0表示只有主进程


In [15]:
import torch.nn as nn
import torch.nn.functional as F

# define NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(54, 2)
        self.fc2 = nn.Linear(54, 54)
        self.fc3 = nn.Linear(54, 54)
    def forward(self,x):
        #x = F.sigmoid(self.fc3(x))
        #x = F.sigmoid(self.fc2(x))
        x = F.sigmoid(self.fc1(x))
        return x

# initialize the NN
model = Net()
#print(model)

In [16]:
print(sum(y24))

tensor([369.])


In [17]:
# specify loss function
#criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss(weight=Weights)
# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

In [18]:
# number of epochs to train the model
n_epochs = 400
train_loss_min = np.Inf  # set initial "min" to infinity
train_loss_curve = list()
train_acc = list()
train_acc1 = list()
train_acc0 = list()
test_loss_curve = list()
test_acc = list()
test_acc1 = list()
test_acc0 = list()

for epoch in range(n_epochs):
    train_loss = 0
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    # train the model #
    loss = 0
    for data, target in train_dataloader: 
        optimizer.zero_grad()
        output = model(data)
        #make target a tensor:torch.Size([10]),dtype = long
        target = target.view(batch_size)
        target = target.long()
        #target = target.T
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1)
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        for i in range(len(target)):
            if (target.data[i]>0.5):
                label = 1
            else:
                label = 0
        #label = torch.tensor(target.data[i], dtype=torch.int64)
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    train_acc.append(np.sum(class_correct) / np.sum(class_total))
    train_acc0.append(class_correct[0] / class_total[0])
    train_acc1.append(class_correct[1] / class_total[1])


    # calculate average loss over an epoch
    train_loss = train_loss / len_train
    train_loss_curve.append(train_loss)
    #print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
    
    # save model
    if train_loss <= train_loss_min:
        #print('Train loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(train_loss_min,train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss

    test_loss = 0.0
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))

    for data, target in test_dataloader:
        output = model(data)
        target = target.view(batch_size)
        target = target.long()
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)
        pred = output.argmax(dim=1)
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        for i in range(len(target)):
            if (target.data[i]>0.5):
                label = 1
            else:
                label = 0
        #label = torch.tensor(target.data[i], dtype=torch.int64)
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    test_acc.append(np.sum(class_correct) / np.sum(class_total))
    test_acc0.append(class_correct[0] / class_total[0])
    test_acc1.append(class_correct[1] / class_total[1])
    test_loss = test_loss/len_test
    test_loss_curve.append(test_loss)


figure, axes = plt.subplots( nrows=1, ncols=2, figsize=(12,6), dpi=80 )
axes[0].plot(np.linspace(1,n_epochs,n_epochs),train_loss_curve,label = 'Training Loss')
axes[1].plot(np.linspace(1,n_epochs,n_epochs),test_loss_curve,label = 'Test Loss')
axes[1].plot(np.linspace(1,n_epochs,n_epochs),test_acc, label = 'Test Accuracy')
axes[1].plot(np.linspace(1,n_epochs,n_epochs),test_acc0, label = 'Test Accuracy of 0')
axes[1].plot(np.linspace(1,n_epochs,n_epochs),test_acc1, label = 'Test Accuracy of 1')
axes[0].plot(np.linspace(1,n_epochs,n_epochs),train_acc, label = 'Train Accuracy')
axes[0].plot(np.linspace(1,n_epochs,n_epochs),train_acc0, label = 'Train Accuracy of 0')
axes[0].plot(np.linspace(1,n_epochs,n_epochs),train_acc1, label = 'Train Accuracy of 1')
axes[0].set_title('Training')
axes[0].set_xlabel('Epoch')
axes[0].set_ylim(0,1.2)
axes[0].legend()
axes[1].set_title('Test')
axes[1].set_xlabel('Epoch')
axes[1].set_ylim(0,1.2)
axes[1].legend()
plt.show()

In [19]:
model.load_state_dict(torch.load('model.pt'))
print(model.fc1.weight)

test_loss = 0.0
correct = 0
OUTPUT = []

for i in range(len_test):
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(X_test[i])
    o = output.tolist()
    OUTPUT.append(o)



Parameter containing:
tensor([[-0.1541,  0.0567, -1.6825,  0.8680, -1.2610,  1.5911,  0.3007, -0.6345,
         -1.0280,  0.8138,  1.6692,  0.6326,  0.1020, -0.2291, -1.1236,  0.2399,
          0.7649, -0.5503,  1.4556,  0.3657,  2.5097, -0.7462,  0.6857, -0.4155,
         -1.4397, -0.7064,  0.7606,  0.6721,  0.1933, -0.1940, -0.0503, -0.0370,
          0.7039,  0.5121,  0.0787,  0.8804, -0.5770, -0.4303, -0.2875, -0.3388,
         -0.9509, -0.7040,  0.4445, -0.2268, -0.8619, -0.2291, -0.1548, -0.4938,
          0.5465,  0.4532,  0.7685, -1.2181,  0.1032, -1.5647],
        [-0.4876, -1.4785,  0.9780, -0.5231,  0.7053, -0.2538, -0.1522, -0.5169,
          1.6113,  0.9185, -1.1391,  0.0566, -0.9124,  0.3475,  1.1779,  0.4361,
          0.7574, -1.1909, -0.4516, -0.7383, -1.9337,  0.9135, -0.6375,  0.5029,
          0.6368,  0.0990, -0.5231,  0.6203, -1.0008,  0.6553, -0.0171, -0.2496,
         -0.0438,  0.3282, -0.2956, -1.1092,  0.6035,  0.2158, -0.0766,  0.8010,
          0.5487, -0.11

In [20]:
for i in range(2):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of     0: 64% (200/309)
Test Accuracy of     1: 49% (95/191)

Test Accuracy (Overall): 59% (295/500)
